In [1]:
from os import walk
from os.path import join
from sklearn.utils import shuffle

path = '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech'
mypath = path
flac = []
txt = []
# 遞迴列出所有檔案的絕對路徑
for root, dirs, files in walk(mypath):
    for f in files:
        if f[-4:] == 'flac':
            fullpath = join(root, f)
            flac.append(fullpath)
        elif f[-4:] == '.txt':
            fullpath = join(root, f)
            txt.append(fullpath)

In [2]:
flac[:5]

['/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0026.flac',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0005.flac',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0033.flac',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0006.flac',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0018.flac']

In [3]:
txt[:5]

['/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/librispeech-lm-norm.txt',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896.trans.txt',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0026.trans.txt',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0005.trans.txt',
 '/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896-0033.trans.txt']

In [9]:
with open(txt[1],'r') as fp:
     all_lines = fp.readlines()

In [12]:
txt[1]

'/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/dev-clean/2277/149896/2277-149896.trans.txt'

In [11]:
all_lines[0]

"2277-149896-0000 HE WAS IN A FEVERED STATE OF MIND OWING TO THE BLIGHT HIS WIFE'S ACTION THREATENED TO CAST UPON HIS ENTIRE FUTURE\n"

In [4]:
from tqdm import tqdm_notebook as tqdm

txt_for_flac = []
for i in tqdm(range(len(flac))):
    path = '/'.join(flac[i].split('/')[:-1])
    filename = '-'.join(flac[i].split('/')[-1].split('-')[:-1])+'.trans.txt'
    flac_filename = flac[i].split('/')[-1].split('.')[0]
    with open(path + '/' +filename,'r') as fp:
         all_lines = fp.readlines()
    for j in range(len(all_lines)):
        if flac_filename in all_lines[j]:
            #print(' '.join(all_lines[j].split(' ')[1:]))
            with open(path + '/' +flac_filename+'.trans.txt','w') as wf:
                wf.write(' '.join(all_lines[j].split(' ')[1:]))
            txt_for_flac.append(path + '/' +flac_filename+'.trans.txt')

In [5]:
for i in tqdm(range(len(txt_for_flac))):
    if flac[i].split('/')[-1].split('.')[0] != txt_for_flac[i].split('/')[-1].split('.')[0]:
        print(flac[i])
        print(txt_for_flac[i])

In [53]:
import pandas as pd
LibriSpeech = pd.DataFrame([flac,txt_for_flac]).T
LibriSpeech_shuffle = shuffle(LibriSpeech).reset_index(drop=True)
LibriSpeech_shuffle[0][0]

'/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/548/126959/548-126959-0059.flac'

In [54]:
L = len(LibriSpeech_shuffle)
train = LibriSpeech_shuffle[:int(L*0.7)].reset_index(drop=True)
val = LibriSpeech_shuffle[int(L*0.7):int(L*0.8)].reset_index(drop=True)
test = LibriSpeech_shuffle[int(L*0.8):].reset_index(drop=True)
name = 'LibriSpeech'
train.to_csv(name + '_train.csv',index=False,header=None)
val.to_csv(name + '_val.csv',index=False,header=None)
test.to_csv(name + '_test.csv',index=False,header=None)

In [56]:
train[0][0]

'/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/548/126959/548-126959-0059.flac'

In [57]:
train[1][0]

'/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/548/126959/548-126959-0059.trans.txt'

In [6]:
text = []
for i in tqdm(range(len(txt_for_flac))):
    fp = open(txt_for_flac[i], "r")
    lines = fp.readlines()
    fp.close()
    text.append(lines)

In [7]:
text_str = ''
for i in tqdm(range(len(text))):
    for j in text[i]:
        text_str += j

In [8]:
unique_word = ''.join(set(text_str))

In [9]:
ner_labels = []
for i in unique_word:
    ner_labels.append(i)

In [11]:
import json

data = ner_labels
ret = json.dumps(data)

with open('../labels/LibriSpeech_ner_labels.json', 'a') as fp:
    fp.write(ret)

In [ ]:
import torchaudio
for i in tqdm(range(len(test[0][:5]))):
    try:
        path = test[0][i]
        sound, _ = torchaudio.load(path, normalization=True)
    except:
        print('='*50)
        print(path)
        print('='*50)

/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-100/405/130894/405-130894-0052.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/7569/102240/7569-102240-0042.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/2769/176263/2769-176263-0030.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-100/2289/152258/2289-152258-0040.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/6782/61316/6782-61316-0016.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/594/127731/594-127731-0003.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/8401/280380/8401-280380-0059.flac
/media/volume1/Eason/Speech_Recognition/End-to-end-ASR-Pytorch/data/LibriSpeech/train-clean-360/